In [1]:
import pandas as pd
from sklearn import preprocessing   #导出数据预处理包
import numpy as np    #导入np数据
import joblib
import pymysql
import json
#导入模型
km=joblib.load(filename='km.model')

### tip:单个dataframe

In [2]:
data={'Chinese':[25],'Math':[100]}
df=pd.DataFrame(data)

In [3]:
labels3 = km.predict(df)
print(labels3)

[2]


## 数据库

In [4]:
def sop():#启动连接
    return pymysql.connect(host="localhost",user="root",passwd="nzzfl2",db="python",port=3306)#数据库连接

In [5]:
def ins(sjk,sql):#插入sql
    c=sjk.cursor()
    try:
        c.execute(sql)
        sjk.commit()
    except:
        sjk.rollback()

In [6]:
def sel(sjk,sql):#查询sql
    c=sjk.cursor()
    try:
        c.execute(sql)
        return c.fetchall()
    except:
        sjk.rollback()
        return None

In [7]:
def stop(sjk):#关闭数据库
    sjk.close()

## 学生成绩

In [8]:
#聚类学生成绩并保存至数据库
def pre(iris_data):
    if 'sno' in iris_data.columns and 'Chinese' in iris_data.columns and 'Math' in iris_data.columns: #上传文件检验
        iris_data.dropna(inplace = True)#去除有空值的数据
        #去除异常数据
        iris_data[iris_data<0]=0
        #iris_data[iris_data>100]=0
        iris_data_2=iris_data[['Chinese','Math']]
        kmpre_data = km.predict(iris_data_2)
        sjkobj=sop()
        try:
            length=len(kmpre_data)
            for i in range(0,length):
                iris=iris_data.iloc[i]
                ins(sjkobj,"INSERT INTO stu(sno,Chinese,Math,type) VALUES('{}','{}','{}','{}')".format(iris[0],iris[1],iris[2],kmpre_data[i]))
            stop(sjkobj)
            return str(kmpre_data[0]) #强转成字符串 服务器返回值不能是int
        except:
            return 'fail'
    else:
        return None

In [9]:
#上传文件
def kmpre(file):
    return pre(pd.read_csv('upload/'+file,sep = ','))

In [10]:
#预测单个学生
def stupre(data):
    if data.empty: #DataFrame判断是否为空
        return 'fail'
    else:
        try:
            data = data.astype(np.int64) #强制转换为int类型
            return pre(data)
        except:
            return 'fail'

In [11]:
#查询
def selectstu(token,inf):
    sjkobj=sop()
    data=None
    if token == '1': #查询全体
        data=sel(sjkobj,"select * from stu")
    if token == '2': #查询单个
        data=sel(sjkobj,"select * from stu where sno="+inf)
    if token == '3': #查询类别
        data=sel(sjkobj,"select * from stu where type="+inf)
    if data != None:
        stop(sjkobj)
        return stujson(data)

def stujson(data): #学生json
    jsonData=[]
    for row in data:
        result={}
        result['sno']=row[0]
        result['Chinese']=row[1]
        result['Math']=row[2]
        result['type']=row[3]
        jsonData.append(result)
    data={}
    data['type']='success'
    data['data']=jsonData
    return data

## 服务器

In [ ]:
import os
from flask import Flask, request, redirect, url_for,jsonify
from werkzeug.utils import secure_filename
#服务器配置
app = Flask(__name__)
UPLOAD_FOLDER = 'upload' #上传文件目录
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['JSON_AS_ASCII'] = False

#单个学生
@app.route('/stu',methods = ['POST', 'GET']) 
def stu():
    if request.method == 'POST':
         return  stupre(pd.DataFrame({'sno':[request.form['sno']],'Chinese':[request.form['Chinese']],'Math':[request.form['Math']]}))
    else:
         return '请使用post传递参数'
        
#上传文件      
@app.route('/uploader',methods=['GET','POST'])
def uploader():
    if request.method == 'POST':
        f = request.files['file']
        f.save(os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(f.filename)))
        txt=kmpre(f.filename)
        if  txt != 'fail':
            if txt == None:
                return '文件上传失败,请检查文件是否符合规范'
            else: 
                return '文件上传成功'
        else:
            return '文件上传失败'
    else:
        return '请使用post传递参数'
    
#查询学生     
@app.route('/selstu',methods=['GET','POST'])
def selstu():
    if request.method == 'POST':
        return jsonify(selectstu(request.form['token'],request.form['information']))
    else:
        return selectstu(1,0)#默认返回全部
    
#服务器运行配置
if __name__ == '__main__':
    app.run(host='127.0.0.1',port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2021 10:00:36] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 10:00:36] "GET /favicon.ico HTTP/1.1" 404 -
